<a href="https://colab.research.google.com/github/fasthill/ML-DL-study-alone/blob/main/5-1%20%EA%B2%B0%EC%A0%95%20%ED%8A%B8%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analyse from LightGBM

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/rickiepark/hg-mldl/blob/master/5-1.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce

In [2]:
from xgboost import XGBClassifier
import xgboost
from lightgbm import LGBMClassifier
import lightgbm

In [4]:
from sklearn.linear_model import LogisticRegression, SGDClassifier, SGDRegressor
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [6]:
def predict_p(test_target, y_predict_list): 
    ps = precision_score(test_target, y_predict_list)
    rs = recall_score(test_target, y_predict_list)
    fs = f1_score(test_target, y_predict_list)
    roc = roc_auc_score(test_target, y_predict_list)
    collect_list = [ps, rs, fs, roc]
    return collect_list

In [ ]:
code = {'005930' : ['삼성전자', 'sec'], '373220' : ['LG에너지솔루션', 'lgenergy'], 
        '000660' : ['SK하이닉스', 'skhinix'], '207940' : ['삼성바이오로직스', 'ssbio'],
        '006400' : ['삼성SDI', 'sdi'], '051910' : ['LG화학', 'lgchemical'],
        '005935' : ['삼성전자우', 'secpre'], '005380' : ['현대차', 'hyunmotor'],
        '035420' : ['NAVER', 'naver'], '000270' : ['기아','kia'],
        '035720' : ['카카오', 'kakao'], '005490' : ['POSCO홀딩스', 'poscoholding'],
        '105560' : ['KB금융', 'kbbank'], '028260' : ['삼성물산', 'sscnt'],
        '068270' : ['셀트리온', 'celltrion'], '012330' : ['현대모비스', 'mobis'],
        '055550' : ['신한지주', 'shgroup'], '066570' : ['LG전자', 'lgelec'],
        '003670' : ['포스코케미칼', 'poscochemical'], '096770' : ['SK이노베이션', 'skinnovation'],
        '033780' : ['KT&G', 'ktng']}

code = {'005930' : ['삼성전자', 'sec']}

In [7]:
fname = f'df_shgroup_sel.pkl'
directory_for_ml = '../data/data_for_ml/'
f_name = directory_for_ml + fname
df = pd.read_pickle(f_name) 

# train, val,: 8, test: 2
split_ratio = 0.8
split_n = int(len(df)*split_ratio)

data = df.iloc[:split_n, :-5]
target = df.iloc[:split_n, -4]
test_input = df.iloc[split_n:, :-5]
test_target = df.iloc[split_n:, -4]

train_input, val_input, train_target, val_target = \
     train_test_split(data, target, random_state=42, test_size=0.2, stratify=target)

ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
val_scaled = ss.transform(val_input)
test_scaled = ss.transform(test_input)

In [ ]:
params_test= { 
    "num_iterations": [100, 200, 300, 400, 500],
    "num_class": [1]
    "metric" : "rmse",
    "bagging_frequency" : 5,
    "bagging_seed" : 2018,
    "verbosity" : -1,

    # Selected rounded-off params
    'bagging_fraction': 0.7,
    'feature_fraction': 0.1,
    'lambda_l1': 1,
    'lambda_l2': 0,
    'max_depth': 9,
    'min_child_weight': 5,
    'min_split_gain': 0,
    'num_leaves': 24 }

In [10]:
params_o = {
    "boosting_type" : ['gbdt'],
    "max_depth": [None, 2, 3, 4],
    "num_leaves": [2, 3, 4],
#     num_leaves 는 10 (왜냐면 작은 데이터이기 때문)
    "learning_rate": [ 0.006, 0.007, 0.008, 0.01],
    "objective": ['binary'],
#     "metric": ['loss'],
    "metric": ['binary_logloss'],
    "colsample_bytree": [ 0.34, 0.35, 0.36, 0.37, 0.38],
    "subsample": [None, 0.001, 0.002],
    "n_estimators": [None, 3, 4, 5],
#     min_data_in_leaf : 큰 값으로 세팅하는 것은 Tree가 너무 깊게 확장되는 것을 막을 수 있지만 
#     under-fitting 언더 피팅이 발생할 수도 있습니다.
    
}

lgbm = None
lgbmgs = None

lgbm = lightgbm.LGBMClassifier( random_state=42, 
                               learning_rate =0.22, # default = 0.1
                               num_iterations=1000, # default=100
                               max_depth=5, # <= 0 means no limit, default=-1
#                                bagging_fraction=0.9, # 0.0 < bagging_fraction <= 1.0, default=1
#                                feature_fraction=0.8, # 0.0 < feature_fraction <= 1.0, default=1
                               objective= 'binary', 
#                                num_threads = 0, # == nthread=4, # Gpu 수, default=0, 자동 검색후 적용
                               max_delta_step = 0.5 # best value found, default = 0
#                                scale_pos_weight=40, # class imbalance 경감, scale_pos_weight > 0.0, default=1.0
                               
                                )

lgbmgs = GridSearchCV(estimator = lgbm,
                      param_grid = params_o,
                      cv = 5, # StratifiedKFold us default for binary or multiclass
#                       scoring = 'precision', 
                      scoring = 'accuracy', 
                      verbose = 1,
                      n_jobs=-1, # 자동 검색 적용
                      )

lgbmgs.fit(train_scaled, train_target, eval_metric = 'logloss')
    
print("Best Estimator: {}".format(lgbmgs.best_estimator_)) 
print("Best Parameters: {}".format(lgbmgs.best_params_))  # 최적 파라미터.
print('Best Score: {}, Best Index: {}'.format(lgbmgs.best_score_ , lgbmgs.best_index_))  # 교차검증된 점수를 보여줌.

model = lgbmgs.best_estimator_  # 최적의 파라미터로 모델 생성
y_pred = model.predict(test_scaled)

train_score = model.score(train_scaled, train_target)
val_score = model.score(val_scaled, val_target)
test_score = model.score(test_scaled, test_target)
ps, rs, fs, roc = predict_p(test_target, model.predict(test_scaled))
cm = confusion_matrix(test_target, model.predict(test_scaled))

print("train accuracy: {:.4f}, val accuracy {:.4f}, test accuracy {:.4f}".
      format(train_score, val_score, test_score))
print("precision : {:.4f}, recall : {:.4f}, f1score : {:.4f}, roc : {:.4f}".
     format (ps, rs, fs, roc))
print(cm)

Fitting 5 folds for each of 2880 candidates, totalling 14400 fits
Best Estimator: LGBMClassifier(colsample_bytree=0.34, learning_rate=0.006, max_delta_step=0.5,
               max_depth=None, metric='loss', n_estimators=None,
               num_iterations=1000, num_leaves=2, objective='binary',
               random_state=42, subsample=None)
Best Parameters: {'boosting_type': 'gbdt', 'colsample_bytree': 0.34, 'learning_rate': 0.006, 'max_depth': None, 'metric': 'loss', 'n_estimators': None, 'num_leaves': 2, 'objective': 'binary', 'subsample': None}
Best Score: 0.798913043478261, Best Index: 0
train accuracy: 0.8739, val accuracy 0.7667, test accuracy 0.7895
precision : 0.8182, recall : 0.6000, f1score : 0.6923, roc : 0.7565
[[21  2]
 [ 6  9]]


C:\Users\user\AppData\Roaming\Python\Python39\site-packages\lightgbm\engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


In [ ]:
model.feature_importances_

In [ ]:
df_sel = pd.DataFrame(model.feature_importances_, index=data.columns, columns=['importance']).sort_values(by='importance', ascending=False)
df_sel

In [ ]:
model.booster_.feature_importance(importance_type='gain')

In [ ]:
df_sel = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), 
                      index=data.columns, columns=['importance']).sort_values(by='importance', 
                                                                              ascending=False)
df_sel

In [ ]:
df_sel = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), 
                      index=data.columns, columns=['importance']).sort_values(by='importance', 
                                                                              ascending=False)
df_sel

In [ ]:
model.booster_.feature_importance(importance_type='split')
# 큰 특징을 가지는 feature는 tree상위레벨에서 적게 사용됨.

In [ ]:
model.booster_.feature_importance(importance_type='gain')

In [ ]:
for i in ['mean_test_score', 'std_test_score']:
        print(i," : ", gsearch1.cv_results_[i])

In [18]:
lgbmgs.best_estimator_

LGBMClassifier(colsample_bytree=0.34, learning_rate=0.006, max_delta_step=0.5,
               max_depth=None, metric='loss', n_estimators=None,
               num_iterations=1000, num_leaves=2, objective='binary',
               random_state=42, subsample=None)

In [183]:
def make_df_from_estimator(estimator, num):
    df_t = pd.DataFrame.from_dict(estimator, orient='index')
    df_t.columns = [f'value{num}']
    df_t.index.name = 'parameter'
    return df_t

In [155]:
# # 최초의 empty df 생성
# df_base = pd.DataFrame(pd.Series([],dtype=pd.StringDtype(), name='parameter')).set_index('parameter')

In [187]:
df_estimator = make_df_from_estimator(lgbmgs.best_estimator_.get_params(), 1)

In [188]:
df_result = make_df_from_estimator(result_dict, 1)

In [ ]:
df_concat = pd.concat([df_estimator, df_result])

In [157]:
# df_base = df_base.merge(df_estimator, how='outer', left_index=True, right_index=True)

In [ ]:
# df_base = df_base.merge(df_estimator, how='outer', left_index=True, right_index=True)

In [189]:
# dfs = [df_estimator, df_result]
# df_merged= reduce(lambda  left,right: pd.merge(left,right, how='outer', left_index=True, right_index=True), dfs)

In [191]:
pd.concat([df_estimator, df_result])

,value1
parameter,
boosting_type,gbdt
class_weight,None
colsample_bytree,0.34
importance_type,split
learning_rate,0.006
max_depth,None
min_child_samples,20
min_child_weight,0.001
min_split_gain,0.0


In [ ]:
print("Best Estimator: {}".format(lgbmgs.best_estimator_)) 
print("Best Parameters: {}".format(lgbmgs.best_params_))  # 최적 파라미터.
print('Best Score: {}, Best Index: {}'.format(lgbmgs.best_score_ , lgbmgs.best_index_))  # 교차검증된 점수를 보여줌.

model = lgbmgs.best_estimator_  # 최적의 파라미터로 모델 생성
y_pred = model.predict(test_scaled)

train_score = model.score(train_scaled, train_target)
val_score = model.score(val_scaled, val_target)
test_score = model.score(test_scaled, test_target)
ps, rs, fs, roc = predict_p(test_target, model.predict(test_scaled))
cm = confusion_matrix(test_target, model.predict(test_scaled))

train accuracy: 0.8739, val accuracy 0.7667, test accuracy 0.7895
precision : 0.8182, recall : 0.6000, f1score : 0.6923, roc : 0.7565
[[21  2]
 [ 6  9]]

In [ ]:
Best Score: 0.798913043478261, Best Index: 0
train accuracy: 0.8739, val accuracy 0.7667, test accuracy 0.7895
precision : 0.8182, recall : 0.6000, f1score : 0.6923, roc : 0.7565
[[21  2]
 [ 6  9]]

In [176]:
model = lgbmgs.best_estimator_  # 최적의 파라미터로 모델 생성
y_predict = model.predict(test_scaled)
result_dict= {}
result_dict['best_score'] = lgbmgs.best_score_ 
result_dict['best_index'] = lgbmgs.best_index_
result_dict['acc_train'] = model.score(train_scaled, train_target)
result_dict['acc_val'] = model.score(val_scaled, val_target)
result_dict['acc_test'] = model.score(test_scaled, test_target)
result_dict['precision'] = precision_score(test_target, y_predict)
result_dict['recall'] = recall_score(test_target, y_predict)
result_dict['f1score'] = f1_score(test_target, y_predict)
result_dict['roc'] = roc_auc_score(test_target, y_predict)
cm = confusion_matrix(test_target, y_predict)
result_dict['tn'] = cm[0,0]
result_dict['fp'] = cm[0,1]
result_dict['fn'] = cm[1,0]
result_dict['tp'] = cm[1,1]

In [184]:
df_result = make_df_from_estimator(result_dict, 1)

In [185]:
df_result

,value1
parameter,
best_score,0.798913
best_index,0.000000
acc_train,0.873950
acc_val,0.766667
acc_test,0.789474
precision,0.818182
recall,0.600000
f1score,0.692308
roc,0.756522


In [ ]:
    ps = precision_score(test_target, y_predict_list)
    rs = recall_score(test_target, y_predict_list)
    fs = f1_score(test_target, y_predict_list)
    roc = roc_auc_score(test_target, y_predict_list)
    collect_list = [ps, rs, fs, roc]

In [159]:
lgbmgs.best_score_ 

0.798913043478261

In [170]:
model.best_score_.values().

dict_values([])

In [171]:
cm = confusion_matrix(test_target, model.predict(test_scaled))

In [172]:
cm[0,0]

21

In [174]:
y_predict = model.predict(test_scaled)

In [175]:
y_predict

array([0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0], dtype=int64)

파라미터 튜닝 :

데이터 사이언티스트는 언제나 어떤 파라미터를 언제 사용할지 그리고 어떤 값이 최적의 파라미터 값일지 결정하기 위해 고민합니다.

아래 소개되는 기법들은 모델 정확도를 향상시키기 위해 사용될 수 있습니다.

num_leaves : Tree 모델의 복잡성을 컨트롤하는 주요 파라미터입니다. 이상적으로 num_leaves 값은 2 ^ (max_depth) 값보다 적거나 같아야 합니다. 이것보다 많은 값은 과적합을 유발할 것입니다.
min_data_in_leaf : 큰 값으로 세팅하는 것은 Tree가 너무 깊게 확장되는 것을 막을 수 있지만 under-fitting 언더 피팅이 발생할 수도 있습니다. 관행적으로, 수백 또는 수천 개로 정하는 것이 큰 데이터 세트에 충분합니다.
max_depth : Tree 깊이를 명확하게 제한하기 위해 max_depth 값을 설정할 수도 있습니다.
더 빠른 속도를 위하여 :

bagging_fraction과 baggin_freq 을 설정하여 bagging 을 적용하십시오
feature_fraction을 설정하여 feature sub-sampling을 하십시오
작은 max_bin 값을 사용하십시오
save_binary 를 값을 통해 다가오는 학습에서 데이터 로딩 속도를 줄이십시오
parallel learning 병렬 학습을 적용하십시오
더 나은 정확도를 위해 :

큰 max_bin 값을 사용하십시오 (아마 속도는 느려질 수 있습니다)
작은 learning_rate 값을 큰 num_iterations 값과 함께 사용하십시오
큰 num_leaves 값을 사용하십시오 (아마 과적합을 유발할 수도 있습니다)
더 큰 트레이닝 데이터를 사용하십시오
dart 를 사용하십시오
범주형 feature를 사용하십시오
과적합을 해결하기 위해 :

작은 max_bin 값을 사용하십시오
작은 num_leaves 값을 사용하십시오
min_data_in_leaf 와 min_sum_hessian_in_leaf 파라미터를 사용하십시오
bagging_fraction 과 bagging_freq 을 사용하여 bagging 을 적용하십시오
feature_fraction을 세팅하여 feature sub-sampling을 하십시오
lambda_l1, lambda_l2 그리고 min_gain_to_split 파라미터를 이용해 regularization (정규화) 를 적용하십시오
max_depth 를 설정해 Deep Tree 가 만들어지는 것을 방지하십시오